In [1]:
from xgboost import XGBRegressor
from causalml.dataset import synthetic_data
from causalml.inference.meta import BaseTClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [2]:
import pandas as pd
df=pd.read_csv('data.csv',index_col=False)
df.head()

,sex,yob,g2000,g2002,g2004,p2000,p2002,p2004,treatment,cluster,voted,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean
0,male,1941,yes,yes,yes,no,yes,No,Civic Duty,1,No,1.0,2,21,0.095238,0.857143
1,female,1947,yes,yes,yes,no,yes,No,Civic Duty,1,No,1.0,2,21,0.095238,0.857143
2,male,1951,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
3,female,1950,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
4,female,1982,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143


In [3]:
df.loc[df['sex']=='male', 'sex']=0
df.loc[df['sex']=='female', 'sex']=1
df['age']=2019-df['yob']
for item in ['g2000','g2002','p2000','p2002','p2004','voted']:
    df.loc[df[item]=='yes', item]=1
    df.loc[df[item]=='Yes', item]=1
    df.loc[df[item]=='no', item]=0
    df.loc[df[item]=='No', item]=0

In [4]:
df = df[(df['treatment']==' Control') | (df['treatment']==' Neighbors')]
df.loc[df['treatment']==' Control', 'treatment']=0
df.loc[df['treatment']==' Neighbors', 'treatment']=1

In [5]:
df.head()

,sex,yob,g2000,g2002,g2004,p2000,p2002,p2004,treatment,cluster,voted,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean,age
5,0,1981,0,0,yes,0,0,0,0,1,0,3.0,3,21,0.047619,0.904762,38
6,1,1959,1,1,yes,0,1,0,0,1,1,3.0,3,21,0.047619,0.904762,60
7,0,1956,1,1,yes,0,1,0,0,1,1,3.0,3,21,0.047619,0.904762,63
8,1,1968,0,0,yes,0,1,0,0,1,0,4.0,2,21,0.047619,0.904762,51
9,0,1967,1,1,yes,0,1,0,0,1,0,4.0,2,21,0.047619,0.904762,52


In [6]:
X=np.array(df[['sex','age','g2000','g2002','p2000','p2002','p2004']], dtype=np.int)
y=np.array(df['voted'],dtype=np.int)
treatment=np.array(df['treatment'],dtype=np.int)

In [7]:
print(X[treatment==1].shape,X[treatment==0].shape)

(38201, 7) (191243, 7)


In [24]:
class BaseTClassifier_(object):
    def __init__(self,learner0,learner1):
        self.learner0 = learner0
        self.learner1 = learner1
        self.model0 = None
        self.model1 = None
        
    def fit(self,X,y,treatment):
        X0 = X[treatment==0]
        X1 = X[treatment==1]
        y0 = y[treatment==0]
        y1 = y[treatment==1]
        
        self.model0 = self.learner0.fit(X0, y0)
        self.model1 = self.learner1.fit(X1, y1)
        
    def predict(self,X, idx):
        if idx==0:
            prob = self.model0.predict_proba(X)[:, 1]
        else:
            prob = self.model1.predict_proba(X)[:, 1]
        return prob
    
    def get_cate(self,X,y,treatment):
        # replace all with estimated value
        self.fit(X,y,treatment)
        prob0 = self.predict(X, idx=0)
        prob1 = self.predict(X, idx=1)
        return prob1-prob0
    
    def get_cate_(self,X,y,treatment):
        # not replace the existing value
        self.fit(X,y,treatment)
        prob0 = self.predict(X[treatment==1], idx=0)
        prob1 = self.predict(X[treatment==0], idx=1)
        
        yhat_0 = np.zeros(len(y))
        yhat_1 = np.zeros(len(y))
        yhat_0[treatment==0]=y[treatment==0]
        yhat_0[treatment==1]=prob0
        yhat_1[treatment==1]=y[treatment==1]
        yhat_1[treatment==0]=prob1
        
        return yhat_1-yhat_0

In [34]:
learner_t_ = BaseTClassifier_(learner0=RandomForestClassifier(), learner1=RandomForestClassifier())
cate_t_ = learner_t_.get_cate(X, y, treatment)

In [10]:
learner_t = BaseTClassifier(learner=RandomForestClassifier())
cate_t = learner_t.fit_predict(X=X, treatment=treatment, y=y)

In [11]:
cate_t=cate_t.reshape([229444,])

In [35]:
cate_t_

array([0.06060539, 0.15971288, 0.05091301, ..., 0.112352  , 0.0514249 ,
       0.06786499])

In [31]:
cate_t

array([0.0616839 , 0.16717824, 0.0507599 , ..., 0.11433577, 0.03808358,
       0.04647825])